In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [2]:
from cosmograph_widget import Cosmograph
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
# dir(Cosmograph)

In [3]:
points = pd.read_json('https://gist.githubusercontent.com/Stukova/d2a3bb22c90680b6beaf8a52c3470296/raw/a80fb63f4f4f0135f827c23aff43183de61a153d/points.json', dtype={'id': str})
points.head(1)

,id,index,value,value2,date,time,color,label
0,0,0,-5,116,2022-11-25 03:25:04.851000+00:00,1970-01-01T01:20:59.956Z,#bab0ab,offence


In [4]:
links = pd.read_json('https://gist.githubusercontent.com/Stukova/d2a3bb22c90680b6beaf8a52c3470296/raw/a80fb63f4f4f0135f827c23aff43183de61a153d/links.json', dtype={'sourceidx': str, 'targetidx': str})
links.head(1)

,source,sourceidx,date,time,color,target,targetidx,value
0,0,0,2012-02-08 11:17:27.900000+00:00,1970-01-01T01:06:54.149Z,#4e79a7,1,1,1


In [5]:
cosmo = Cosmograph(points=points, links=links,
   point_id_by='id',
   point_include_columns=['value', 'value2', 'color'],
   point_label_by='label',
   link_source_by='source',
   link_target_by='target',
   link_include_columns=['value'],
)

gravitySlider = widgets.FloatSlider(value=0.25, min=0, max=0.5, step=0.01, description='gravity:')
repulsionSlider = widgets.FloatSlider(value=1, min=0, max=2, step=0.01, description='repulsion:')
linkStrengthSlider = widgets.FloatSlider(value=1, min=0, max=2, step=0.01, description='link strength:')
minimumLinkDistance = widgets.IntSlider(value=10, min=1, max=20, step=0.01, description='min link dist:')

widgets.jslink((gravitySlider, 'value'), (cosmo, 'simulation_gravity'))
widgets.jslink((repulsionSlider, 'value'), (cosmo, 'simulation_repulsion'))
widgets.jslink((linkStrengthSlider, 'value'), (cosmo, 'simulation_link_spring'))
widgets.jslink((minimumLinkDistance, 'value'), (cosmo, 'simulation_link_distance'))

accelerateButton = widgets.Button(description='Accelerate')
accelerateButton.on_click(lambda x: cosmo.start(1))

display(gravitySlider, repulsionSlider, linkStrengthSlider, minimumLinkDistance, accelerateButton, cosmo)

FloatSlider(value=0.25, description='gravity:', max=0.5, step=0.01)

FloatSlider(value=1.0, description='repulsion:', max=2.0, step=0.01)

FloatSlider(value=1.0, description='link strength:', max=2.0, step=0.01)

IntSlider(value=10, description='min link dist:', max=20, min=1, step=0)

Button(description='Accelerate', style=ButtonStyle())

Cosmograph(background_color=None, focused_point_ring_color=None, hovered_point_ring_color=None, link_color=Non…

#### Legends

For each property of point and link styling and sizing such as `point_size_by`, `link_width_by`, `point_color_by`, and `link_color_by`, there is a legend:

In [6]:
cosmo.point_size_by = 'value'
cosmo.link_width_by = 'value'
cosmo.point_color_by = 'value'
cosmo.link_color_by = 'value'

The legend can be disabled, so it does not appear on the graph:

In [7]:
# cosmo.disable_point_size_legend = True
cosmo.disable_link_width_legend = True
cosmo.disable_point_color_legend = True
cosmo.disable_link_color_legend = True

#### Selecting Points

Click on the maximum value in the legend to select the largest points. This will write an array of the selected points into the `selected_point_ids` property.

In [8]:
len(cosmo.selected_point_ids)

74

### Filter points and links by selected data

From the current dataframe, you can create a dataframe of the selected points and render the graph with only the selected points:

In [9]:
selected_points = points.loc[points['id'].isin(cosmo.selected_point_ids)]
selected_points.head(1)

,id,index,value,value2,date,time,color,label
10,10,10,50,7,2018-05-04 17:13:57.021000+00:00,1970-01-01T02:45:56.825Z,#9c755f,congressman


In [10]:
selected_links = links.loc[links.sourceidx.isin(cosmo.selected_point_ids) & links.targetidx.isin(cosmo.selected_point_ids)].copy()
selected_links.head(1)

,source,sourceidx,date,time,color,target,targetidx,value
10159,5107,5107,2012-02-21 11:07:28.486000+00:00,1970-01-01T00:58:44.768Z,#76b7b2,5108,5108,3


In [11]:
cosmo_selected = Cosmograph(points=selected_points, links=selected_links,
   point_id_by='id',
   point_color_by='color',

   link_source_by='source',
   link_target_by='target',
   link_color_by='color'
)
cosmo_selected

Cosmograph(background_color=None, focused_point_ring_color=None, hovered_point_ring_color=None, link_color=Non…

In [12]:
cosmo_selected.fit_view()

In [13]:
cosmo_selected.select_points_by_ids(selected_points.query("color == '#af7aa1'").id.tolist())

In [14]:
cosmo_selected.selected_point_ids

['242', '249', '2714', '3222', '4696', '4763', '5407', '6680', '7260']

In [15]:
cosmo_selected.fit_view_by_ids(cosmo_selected.selected_point_ids, 2000)

In [16]:
cosmo_selected.focus_point('242')

In [17]:
cosmo_selected.focus_point()

In [ ]:
cosmo_selected.activate_rect_selection()

In [ ]:
cosmo_selected.deactivate_rect_selection()